# Gridded forecast tutorial - Italy 2010 Experiment

In this tutorial, we will load and test grid-based forecasts and interpret the results of the tests provided in the PyCSEP package for gridded forecasts. We will work with two time-independent grid-based forecasts submitted as part of the CSEP Italy testing experiment (see [Werner et al, 2010](https://doi.org/10.4401/ag-4840), [Taroni et al, 2018](https://doi.org/10.1785/0220180031) for some previous testing results). Our goal is to compare the performance of these two forecasts for describing observed Italian seismicity.   

This is essentially a three step process:  
    1. Read in (and plot) a gridded forecast  
    2. Set up an evaluation catalog of observed events  
    3. Run PyCSEP tests and interpret the results    


We introduce the concepts to the reader and encourage them to explore the other tests available. Full documentation of the package can be found [here](https://docs.cseptesting.org/) and any issues can be reported on the [PyCSEP Github page](https://github.com/SCECcode/pycsep).

In [ ]:
# Main functionalities are found at the top-level of the csep package
import csep
# The testing core classes and routines are found in the csep.core sub-module
from csep.core import regions, poisson_evaluations
# Utilities are available from the csep.utils sub-module.
from csep.utils import datasets, time_utils, comcat, plots, readers

import numpy
import cartopy

## Read in forecasts

We're going to start by setting up some experiment parameters. It is good practice to set this up early. Note, the start and end date of the forecast should be chosen based on the creation of the forecast. This is important for time-independent forecasts because they can be rescaled to any arbitrary time period.

In [ ]:
## Set up experiment parameters
start_date = time_utils.strptime_to_utc_datetime('2010-01-01 00:00:00.0')
end_date = time_utils.strptime_to_utc_datetime('2015-01-01 00:00:00.0')


We will load two gridded forecasts found in `workshop_data/forecasts` in the standard CSEP `.dat` format. You can learn more about the multiple accepted and custom formats [in the documentation](https://docs.cseptesting.org/concepts/forecasts.html). An example of the `.dat`format is shown below.
```
# min_lat | max_lat | min_lon | max_lon | min_depth | max_depth | min_mag | max_mag | rate | mask
5.50	5.60	44.90	45.00	0.00	30.00	4.95	5.05	3.3000000000000003e-05	1
5.50	5.60	44.90	45.00	0.00	30.00	5.05	5.15	2.5999999999999998e-05	1
5.50	5.60	44.90	45.00	0.00	30.00	5.15	5.25	2.0000000000000002e-05	1
...
5.50	5.60	45.00	45.10	0.00	30.00	4.95	5.05	3.3000000000000003e-05	1
5.50	5.60	45.00	45.10	0.00	30.00	5.05	5.15	2.5999999999999998e-05	1
...
```

High Resolution Smoothed Seismicity Model 1 - [Werner et al., 2010](https://doi.org/10.4401/ag-4839)

In [ ]:
## Loads from the PyCSEP package
werner_forecast = csep.load_gridded_forecast('../workshop_data/forecasts/HRSS_m1.dat',
                                             name='Werner, et al (2010)')

Seismicity forecast from the 2004 Italian Seismic Hazard Model  - [Meletti et al., 2004](http://zonesismiche.mi.ingv.it/)

In [ ]:
## You may need to edit the file location here depending on your set up
meletti_forecast = csep.load_gridded_forecast("../workshop_data/forecasts/MPS04after.dat",
                                              name ="Meletti et al (2010), MPS working group")

Gridded forecasts inherit the region from the forecast, so there is no requirement to explicitly set this. We should check, however, that the forecast regions for catalogs we want to compare are the same so that they are testable with a single catalog. 

In [ ]:
## Sanity check - if forecasts have the same region this will provide no output.
numpy.testing.assert_allclose(meletti_forecast.region.midpoints(), werner_forecast.region.midpoints())

To visualise this forecast, we will use `forecast.plot()` with some specifications to get a nicer looking figure. We will do this by creating a dictionary containing the plot arguments.  

These arguments are, in order:

    - Assign a title
    - Set labels to the geographic axes
    - Draw country borders
    - Set a linewidth of 0.5 to country borders
    - Select ESRI Imagery as a basemap.
    - Assign 'rainbow' as colormap. Possible values from from matplotlib.cm library
    - Defines 0.8 for an exponential transparency function (default is 0 for constant alpha, whereas 1 for linear).
    - An object cartopy.crs.Projection() is passed as Projection to the map  
 
 The complete description of plot arguments can be found in `csep.utils.plots.plot_spatial_dataset`

In [ ]:
args_dict = {'title': 'Italy 10 year forecast',
             'grid_labels': True,
             'borders': True,
             'feature_lw': 0.5,
             'basemap': 'ESRI_imagery',
             'cmap': 'rainbow',
             'alpha_exp': 0.8,
             'projection': cartopy.crs.Mercator()}

The map extent can also be defined. Otherwise, the extent of the data would be used. The dictionary defined must be passed as an argument. 

In [ ]:
ax = werner_forecast.plot(extent=[3, 22, 35, 48],
                          show=True,
                          plot_args=args_dict)

In [ ]:
# Plot the second forecast
# An exercise for the reader
ax = ???

## Set up evaluation catalog

Now we need to import the observed catalog that we want to use to test the forecast - we call this the **evaluation catalog**.  

There are multiple ways to include evaluation catalogs, including accessing directly to catalog providers, such as ComCat for US models, GeoNet for New Zealand, BSI for Italy, etc. There are also various readers currently included with the package, including those for JMA and the INGV HORUS catalogs, which are described [in the documentation](https://docs.cseptesting.org/concepts/catalogs.html#loading-catalogs). These functions are found in `csep/utils/readers.py` if you would like to see them or understand how to add your own.  

In this case we demonstrate by querying the API of the Bollettino Seismico Italiano (INGV).

In [ ]:
## query catalog from provider
italy_test_catalog = csep.query_bsi(start_time = start_date, end_time=end_date)

## In case the INGV API cannot be accesed, load the catalog from the workshop material
# italy_test_catalog = csep.load_catalog("../workshop_data/catalogs/italy_bsi.csv")

Print the catalog to check the range of dates, locations and magnitudes of the events in the evaluation catalog, as well as the total number of events.

In [ ]:
# An exercise for the reader
print(dir(italy_test_catalog))  # print the catalog available methods

We can also filter the catalog to the desired time-space-magnitude range. Crucially, we must also filter the catalog to the forecast region in order to carry out any testing in the next step. This is also why we checked that the forecasts we want to compare are in the same spatial region. 

In [ ]:
italy_test_catalog = italy_test_catalog.filter_spatial(werner_forecast.region)

Note that our magnitude range is not consistent with the parameters we established earlier, so we have to filter for magnitude also. This is obviously important to fairly test a forecast, and you can see why if you re-run this tutorial without this step!

In [ ]:
italy_test_catalog = italy_test_catalog.filter('magnitude >= 4.95')

If you have run the above code, you should be left with a catalog of 13 events. Print the catalog with the standard python `print` command to check the range of dates, locations and magnitudes of the events in the evaluation catalog, as well as the total number of events.

## Run consistency tests

Now we wish to answer some questions about our forecasts and their performance. In this example, we will investigate the spatial properties of the forecast models and how well the forecasts describe the observed spatial distribution of seismicity.

The consistency tests implemented for gridded forecasts in PyCSEP are the N, S and M-test described in [Schorlemmer et al, 2007](https://doi.org/10.1785/gssrl.78.1.17) and [Zechar et al, 2010](https://doi.org/10.1785/0120090192). These are located in the `poisson_evaluations` file that we have imported as `poisson`.

To carry out a test, we simply provide the forecast we wish to test and an evaluation forecast. The spatial test requires simulating from the Poisson forecast to provide uncertainty.

In [ ]:
spatial_test_result_werner = poisson_evaluations.spatial_test(werner_forecast, italy_test_catalog)

In [ ]:
## Repeat the spatial test for our second example forecast
spatial_test_result_meletti = ???

PyCSEP provides easy ways of storing objects to a JSON format using csep.write_json(). The evaluations can be read back into the program for plotting using `csep.load_evaluation_result()`.

In [ ]:
## Run this cell to write to .json file (optional)
## You can look at the contents of this file in jupyter lab to see how the data is stored
csep.write_json(spatial_test_result_meletti, 'example_spatial_test.json')

We can plot these results using the `plot_poisson_consistency_test` function from the `plots` file of `csep.utils`, where you can find more details on the plot arguments or [in the documentation](https://docs.cseptesting.org/reference/generated/csep.utils.plots.plot_poisson_consistency_test.html#csep.utils.plots.plot_poisson_consistency_test). Again, we use a dictionary to set up some plot arguments.

In [ ]:
args = {'figsize': (6,5),
        'title': r'$\mathcal{S}-\mathrm{test}$',
        'title_fontsize': 18,
        'xlabel': 'Log-likelihood',
        'xticks_fontsize': 12,
        'ylabel_fontsize': 12,
        'linewidth': 1,
        'capsize': 4,
        'hbars':True,
        'tight_layout': True}

We're now going to plot the results of both forecasts for comparison. We set `one_sided_lower=True` as usual for an L-test, where the model is rejected if the observed is located within the lower tail of the simulated distribution. We can supply multiple `spatial_test_result` objects in a list (specified in the square brackets as standard in python).

In [ ]:
ax = plots.plot_poisson_consistency_test([spatial_test_result_werner, spatial_test_result_meletti],
                                         one_sided_lower=True, plot_args=args)


We can repeat this process for N or M tests using the `number_test` or `magnitude_test` from `poisson_evaluations` if we are more interested in these components specifically, or getting a fuller picture of where the forecast does well and not so well.   
Try out a `likelihood_test` or `conditional_likelihood_test` (also from `poisson_evaluations`). What does this tell you about the two forecasts?

How should this result be interpreted? Are there other tests that might be useful here? What are your overall conclusions on the performance of these two forecasts?  


If you are happy with all of this so far, you can try:  
* Check the [theory](https://docs.cseptesting.org/getting_started/theory.html) and the [implementation](https://docs.cseptesting.org/concepts/evaluations.html) of additional testing methods 
* Changing the time period for the evaluation    
* Filtering the evaluation catalog by a different time period or magnitude    
* Using PyCSEP gridded-forecast tests on your own forecast models

### References  

Schorlemmer, D., M. Gerstenberger, S. Wiemer, D. D. Jackson, and D. A. Rhoades (2007). Earthquake likelihood model testing, Seismological Research Letters 78 17-29.  
Taroni, M., W. Marzocchi, D. Schorlemmer, M. J. Werner, S. Wiemer, J. D. Zechar, L. Heiniger, F. Euchner (2018). Prospective CSEP Evaluation of 1‐Day, 3‐Month, and 5‐Yr Earthquake Forecasts for Italy, Seismological Research Letters (2018) 89 (4): 1251–1261  
Werner, M. J., J. D. Zechar, W. Marzocchi, S., CSEP-Italy Working Group (2010). Retrospective evaluation of the five-year and ten-year CSEP-Italy earthquake forecasts, Annals of Geophysics, 53, 3.  
Zechar, J. D., M. C. Gerstenberger, and D. A. Rhoades (2010). Likelihood-Based Tests for Evaluating Space-Rate-Magnitude Earthquake Forecasts, Bulletin of the Seismological Society of America 100 1184-1195.